## 선관위의 개표 진행 페이지의 정보 + 등록후보자 API 정보 -> 선거구별 당선자 테이블 생성

In [54]:
from selenium import webdriver 
import time
import pandas as pd
# from selenium.webdriver.common.by import By
import pyautogui
import pyperclip
import urllib.request
from urllib.request import urlretrieve
import requests
from bs4 import BeautifulSoup
import csv
import os
headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (HTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"}
import numpy as np
import xmltodict
import json
import warnings
pd.set_option('display.max_columns', 100)
# Ignore all warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 10000)


In [55]:
df = pd.read_excel('./등록후보자/등록후보자.xlsx')
df.head()

,시도,선거구명,시군구명,정당명,후보자명,등록여부,시도_선거구명
0,서울특별시,종로구,종로구,더불어민주당,곽상언,등록,서울특별시_종로구
1,서울특별시,종로구,종로구,국민의힘,최재형,등록,서울특별시_종로구
2,서울특별시,종로구,종로구,새로운미래,진예찬,등록,서울특별시_종로구
3,서울특별시,종로구,종로구,개혁신당,금태섭,등록,서울특별시_종로구
4,서울특별시,종로구,종로구,가락특권폐지당,김준수,등록,서울특별시_종로구


In [56]:
df_repo = pd.DataFrame(index = df.선거구명.unique())
df_repo.loc['종로구','123'] ='0'

In [57]:
df_sgg = df.groupby(['시도','선거구명'])[['시도','선거구명']].head(1)
df_sgg_0 = df.groupby(['시도','선거구명'])[['정당명']].count().reset_index().rename({'정당명':'정당수'}, axis = 1)
df_sgg_1 =  df_sgg.groupby(['시도'])[['선거구명']].count().reset_index().rename({'선거구명':'선거구수'}, axis = 1)
df_sgg_0.head()

,시도,선거구명,정당수
0,강원특별자치도,강릉시,3
1,강원특별자치도,동해시태백시삼척시정선군,3
2,강원특별자치도,속초시인제군고성군양양군,2
3,강원특별자치도,원주시갑,2
4,강원특별자치도,원주시을,2


In [58]:
df_sgg_0['시도선거구'] = df_sgg_0['시도'] + df_sgg_0['선거구명']

In [59]:
df_sgg_0.columns

Index(['시도', '선거구명', '정당수', '시도선거구'], dtype='object')

In [60]:
df_sgg_0 = df_sgg_0.set_index('선거구명')
df_sgg_0['선거구명'] = df_sgg_0.index

In [61]:
df_sgg_0.head()

,시도,정당수,시도선거구,선거구명
선거구명,,,,
강릉시,강원특별자치도,3,강원특별자치도강릉시,강릉시
동해시태백시삼척시정선군,강원특별자치도,3,강원특별자치도동해시태백시삼척시정선군,동해시태백시삼척시정선군
속초시인제군고성군양양군,강원특별자치도,2,강원특별자치도속초시인제군고성군양양군,속초시인제군고성군양양군
원주시갑,강원특별자치도,2,강원특별자치도원주시갑,원주시갑
원주시을,강원특별자치도,2,강원특별자치도원주시을,원주시을


In [62]:
#소요시간: 5분

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.request import urlretrieve
from selenium.webdriver.chrome.options import Options
from tqdm import tqdm

options = Options()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
driver = webdriver.Chrome(options = options)

# driver.implicitly_wait(10)
li_num = []
li_sgg = []
# 인천광역시 = df_후보자['시도'].unique()[3]
# 대전광역시 = df_후보자['시도'].unique()[5]
# _id = driver.find_element('name','username')
# _id.send_keys(instagram_id)
# time.sleep(2)
li_suc = [] #수집 성공 선거구 리스트 
li_error = [] #수집 실패 선거구 리스트
dic_시도 = {2:'서울특별시',3:'부산광역시',4:'대구광역시',5:'인천광역시',6:'광주광역시',7:'대전광역시',8:'울산광역시',9:'세종특별자치시'
          ,10:'경기도',11:'강원특별자치도',12:'충청북도',13:'충청남도',14:'전북특별자치도',15:'전라남도',16:'경상북도',17:'경상남도',18:'제주특별자치도'}
dic_시도코드 = {}
for i in dic_시도.keys():
    dic_시도코드[dic_시도[i]] = i
df_sgg_1['선거구코드'] = df_sgg_1['시도'].map(dic_시도코드)
df_sgg_0['선거구코드'] = df_sgg_0['시도'].map(dic_시도코드)

# driver = webdriver.Chrome()
driver.get("http://info.nec.go.kr/main/showDocument.xhtml?electionId=0020240410&topMenuId=VC&secondMenuId=VCCP09")
driver.find_element(By.CSS_SELECTOR, '#electionId2').click()
#cityCode

# Set implicit wait
driver.implicitly_wait(5)  # Adjust the time as needed

driver.find_element(By.CSS_SELECTOR, '#cityCode').click()  # 시도 선택 박스켜기

df_repo_cnd = pd.DataFrame(index = df_sgg_0.index)
df_repo_cnd['시도선거구'] = df_sgg_0['시도선거구']
df_repo_vot = pd.DataFrame(index = df_sgg_0.index)
df_repo_vot['시도선거구'] = df_sgg_0['시도선거구']
for i in tqdm(range(2, 19)):  # 시도 선택 레벨
    driver.find_element(By.CSS_SELECTOR, f'#cityCode > option:nth-child({i})').click()  # child(2)-서울 ~ child(18)-제주, 각각 17 시도
    num_sgg = int(df_sgg_1.loc[df_sgg_1['선거구코드'] == i]['선거구수'].values)
    for j in range(2, num_sgg+2):  # 선거구 선택레벨
        driver.find_element(By.CSS_SELECTOR, '#sggCityCode').click()  # 선거구 선택 박스켜기
        driver.find_element(By.CSS_SELECTOR, f'#sggCityCode > option:nth-child({j})').click()  # 선거구 선택
        driver.find_element(By.CSS_SELECTOR, '#spanSubmit > input[type=image]').click()  # 검색클릭
        sgg = driver.find_element(By.CSS_SELECTOR, '#table01 > tbody > tr:nth-child(1) > td:nth-child(1)').text
        #table01 > tbody > tr:nth-child(1) > td:nth-child(1)
        num_prt = int(df_sgg_0.loc[(df_sgg_0['선거구코드'] == i)&(df_sgg_0['선거구명'] == sgg)]['정당수'].values)
        for k in range(5,num_prt+5):
            sido = driver.find_element(By.CSS_SELECTOR, '#cityName').text[1:-1] 
            prt = driver.find_element(By.CSS_SELECTOR,f'#table01 > tbody > tr:nth-child(1) > td:nth-child({k}) > strong').text[:-3]
            cnd = driver.find_element(By.CSS_SELECTOR,f'#table01 > tbody > tr:nth-child(1) > td:nth-child({k}) > strong').text[-3:]
            vot = driver.find_element(By.CSS_SELECTOR, f'#table01 > tbody > tr:nth-child(2) > td:nth-child({k})').text
            df_repo_cnd.loc[df_repo_cnd['시도선거구'] == sido+sgg,prt] = cnd
            df_repo_vot.loc[df_repo_vot['시도선거구'] == sido+sgg,prt] = vot
            df_repo_vot.loc[df_repo_vot['시도선거구'] == sido+sgg,'시도'] = sido
            df_repo_cnd.loc[df_repo_cnd['시도선거구'] == sido+sgg,'시도'] = sido
driver.quit()

100%|██████████| 17/17 [01:52<00:00,  6.59s/it]


In [63]:
driver.quit()

In [64]:
df_repo_cnd.columns =['시도선거구', '더불어민주당', '시도', '국민의힘', '새로운미래', '개혁신당', '가락특권폐지당',
       '대한국민당', '민중민주당', '한국국민당', '무소속', '녹색정의당', '녹색정의당', '국민의힘',
       '더불어민주당', '기후민생당', '진보당', '내일로미래로', '자유통일당', '새진보연합',
       '우리공화당', '자유민주당', '소나무당', '기독당', '노동당', '새로운미래', '개혁신당',
       '한국농어민당', '진보당']
df_repo_vot.columns = ['시도선거구', '더불어민주당', '시도', '국민의힘', '새로운미래', '개혁신당', '가락특권폐지당',
       '대한국민당', '민중민주당', '한국국민당', '무소속', '녹색정의당', '녹색정의당', '국민의힘',
       '더불어민주당', '기후민생당', '진보당', '내일로미래로', '자유통일당', '새진보연합',
       '우리공화당', '자유민주당', '소나무당', '기독당', '노동당', '새로운미래', '개혁신당',
       '한국농어민당', '진보당']

In [65]:
for i in df_repo_vot.columns:
    df_repo_vot[i] = df_repo_vot[i].replace('nan',0)

for i in df_repo_cnd.columns:
    df_repo_cnd[i] = df_repo_cnd[i].replace('nan',0)
df_repo_vot['가락특권폐지당'] = df_repo_vot['가락특권폐지당'].replace('na',0)
df_repo_vot['대한국민당'] = df_repo_vot['대한국민당'].replace('na',0)
df_repo_cnd['가락특권폐지당'] = df_repo_cnd['가락특권폐지당'].replace('na',0)
df_repo_cnd['대한국민당'] = df_repo_cnd['대한국민당'].replace('na',0)


In [31]:
df_repo_cnd.to_excel('temp_cnd.xlsx')
df_repo_vot.to_excel('temp_vot.xlsx')
df_repo_cnd_1 = pd.read_excel('temp_cnd.xlsx', thousands=',').rename({'Unnamed: 0':'선거구명'})
df_repo_vot_1 = pd.read_excel('temp_vot.xlsx', thousands=',').rename({'Unnamed: 0':'선거구명'})

In [32]:
li_col = ['녹색정의당.1', '국민의힘.1', '더불어민주당.1','새로운미래.1', '개혁신당.1','진보당.1']
for col in li_col:
    for i in range(len(df_repo_vot_1)):
        df_repo_vot_1.loc[i,col[:-2]] = df_repo_vot_1.loc[i,col] + df_repo_vot_1.loc[i,col[:-2]]


In [33]:
li_col = ['녹색정의당.1', '국민의힘.1', '더불어민주당.1','새로운미래.1', '개혁신당.1','진보당.1']
for col in li_col:
    for i in range(len(df_repo_cnd_1)):
        if  df_repo_cnd_1.loc[i,col[:-2]] == 0:
            df_repo_cnd_1.loc[i,col[:-2]] = df_repo_cnd_1.loc[i,col]

In [34]:
df_repo_cnd_1.columns

Index(['선거구명', '시도선거구', '더불어민주당', '시도', '국민의힘', '새로운미래', '개혁신당', '가락특권폐지당',
       '대한국민당', '민중민주당', '한국국민당', '무소속', '녹색정의당', '녹색정의당.1', '국민의힘.1',
       '더불어민주당.1', '기후민생당', '진보당', '내일로미래로', '자유통일당', '새진보연합', '우리공화당',
       '자유민주당', '소나무당', '기독당', '노동당', '새로운미래.1', '개혁신당.1', '한국농어민당', '진보당.1'],
      dtype='object')

In [35]:
df_repo_vot_1 = df_repo_vot_1[['시도선거구', '시도', '선거구명', '더불어민주당', '국민의힘','녹색정의당', '새로운미래',
                               '자유통일당','진보당','개혁신당', '민중민주당','한국국민당', '녹색정의당', '기후민생당',
                               '내일로미래로',  '새진보연합','우리공화당', '자유민주당', '소나무당', '기독당', '노동당',
                               '한국농어민당', '무소속']]

In [36]:
df_repo_cnd_1 = df_repo_cnd_1[['시도선거구', '시도', '선거구명', '더불어민주당', '국민의힘','녹색정의당', '새로운미래',
                               '자유통일당','진보당','개혁신당', '민중민주당','한국국민당', '녹색정의당', '기후민생당',
                               '내일로미래로',  '새진보연합','우리공화당', '자유민주당', '소나무당', '기독당', '노동당',
                               '한국농어민당', '무소속']]

In [37]:
df_repo_vot_1['SCS_CNT'] = df_repo_vot_1.iloc[:,3:].max(axis = 1)
df_repo_vot_1['SCS_PRT'] = df_repo_vot_1.iloc[:,3:].idxmax(axis=1)

In [38]:
df_repo_cnd_1['scs_prt'] = df_repo_vot_1['scs_prt']

In [44]:
df_repo_cnd_1['SCS_CD'] = 0
df_repo_vot_1['SCS_CNT'] = 0
for i,prt in zip(range(len(df_repo_cnd_1)), df_repo_cnd_1['scs_prt']):
    df_repo_cnd_1.loc[i, 'SCS_CD'] = df_repo_cnd_1.loc[i, prt]
    df_repo_vot_1.loc[i, 'SCS_CNT'] = df_repo_vot_1.loc[i, prt]


In [251]:
# df_repo_vot_1['선거구'] = df_repo_vot_1['시도'] + df_repo_vot_1['선거구명']
# df_repo_cnd_1['선거구'] = df_repo_cnd_1['시도'] + df_repo_cnd_1['선거구명']
# for sgg in template.index:
#     sg=str(df_repo_cnd_1.loc[df_repo_cnd_1['선거구'] == sgg,'scs_prt'].values)[2:-2]
#     cn=str(df_repo_cnd_1.loc[df_repo_cnd_1['선거구']==sgg,sg].values)[2:-2]
#     template.loc[sgg, '소속정당'] = sg
#     template.loc[sgg, '후보자명'] = cn
# template = template.reset_index()
# template['후보자명'] = template['후보자명'].str.replace('\\n','')